In [1]:
from datasets import load_dataset

# Load the 'google/fleurs' dataset for 'hi_in' (Hindi - India)
dset = load_dataset("SEACrowd/indo_general_mt_en_id", trust_remote_code=True)


/home/fachri/nusax/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Split the dataset into training, validation, and test sets
train_data = dset['train']
val_data = dset['validation']
test_data = dset['test']

In [3]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, SimpleRNN, Embedding, Dense, Attention
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers

# Extract source (Indonesian) and target (English) texts from each split
train_source_texts = [example['src'] for example in train_data][:10000]
train_target_texts = [example['tgt'] for example in train_data][:10000]

val_source_texts = [example['src'] for example in val_data]
val_target_texts = [example['tgt'] for example in val_data]

test_source_texts = [example['src'] for example in test_data]
test_target_texts = [example['tgt'] for example in test_data]

2024-11-10 07:40:09.975490: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-10 07:40:09.997333: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-10 07:40:09.997353: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-10 07:40:09.998179: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-10 07:40:10.002535: I tensorflow/core/platform/cpu_feature_guar

In [4]:
# Add special tokens to your training data
train_target_texts = ["<SOS> " + text + " <EOS>" for text in train_target_texts]
val_target_texts = ["<SOS> " + text + " <EOS>" for text in val_target_texts]
test_target_texts = ["<SOS> " + text + " <EOS>" for text in test_target_texts]


In [5]:
train_target_texts

['<SOS> Pak Tanaka bukan murid. <EOS>',
 '<SOS> Ayah saya guru. <EOS>',
 '<SOS> Sekolah sedang libur. <EOS>',
 '<SOS> Tokyo cerah. <EOS>',
 '<SOS> Di taman ada pohon. <EOS>',
 '<SOS> Pak Tanaka ada di mana? <EOS>',
 '<SOS> Saya tidak ada uang. <EOS>',
 '<SOS> Di atas meja ada buku. <EOS>',
 '<SOS> Di bawah meja ada tas. <EOS>',
 '<SOS> Di dalam tas ada buku tulis. <EOS>',
 '<SOS> Di sebelah kanan Pak Tanaka ada Pak Yamada. <EOS>',
 '<SOS> Di sebelah kiri Pak Tanaka ada Mbak Kimura. <EOS>',
 '<SOS> Di depan rumah ada kucing. <EOS>',
 '<SOS> Di belakang rumah ada mobil. <EOS>',
 '<SOS> Pos surat ada di seberang rumah. <EOS>',
 '<SOS> Pak Yamada ada di luar kelas. <EOS>',
 '<SOS> Di depan rumah ada taman. <EOS>',
 '<SOS> Sekolah ada di dekat rumah. <EOS>',
 '<SOS> Buku di dalam tas. <EOS>',
 '<SOS> Inilah pen saya. <EOS>',
 '<SOS> Kelas bahasa Jepang di mana? <EOS>',
 '<SOS> Itu koran kapan? <EOS>',
 '<SOS> Di kelas ada komputer baru. <EOS>',
 '<SOS> Di dalam kotak ada anjing kecil. <EOS>

In [6]:
# Tokenization (Source and Target)
source_tokenizer = Tokenizer()
target_tokenizer = Tokenizer()

source_tokenizer.fit_on_texts(train_source_texts)
target_tokenizer.fit_on_texts(train_target_texts)

source_vocab_size = len(source_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1

# Convert texts to sequences
train_source_sequences = source_tokenizer.texts_to_sequences(train_source_texts)
train_target_sequences = target_tokenizer.texts_to_sequences(train_target_texts)

val_source_sequences = source_tokenizer.texts_to_sequences(val_source_texts)
val_target_sequences = target_tokenizer.texts_to_sequences(val_target_texts)

test_source_sequences = source_tokenizer.texts_to_sequences(test_source_texts)
test_target_sequences = target_tokenizer.texts_to_sequences(test_target_texts)

In [7]:
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences

# Define maximum sequence lengths (modify as needed)
max_source_length = max(len(seq) for seq in train_source_sequences)
max_target_length = max(len(seq) for seq in train_target_sequences)

# Pad sequences to ensure uniform length
train_source_sequences = pad_sequences(train_source_sequences, maxlen=max_source_length, padding='post')
train_target_sequences = pad_sequences(train_target_sequences, maxlen=max_target_length, padding='post')

val_source_sequences = pad_sequences(val_source_sequences, maxlen=max_source_length, padding='post')
val_target_sequences = pad_sequences(val_target_sequences, maxlen=max_target_length, padding='post')

test_source_sequences = pad_sequences(test_source_sequences, maxlen=max_source_length, padding='post')
test_target_sequences = pad_sequences(test_target_sequences, maxlen=max_target_length, padding='post')

# Create TensorFlow Datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_source_sequences, train_target_sequences))
val_dataset = tf.data.Dataset.from_tensor_slices((val_source_sequences, val_target_sequences))
test_dataset = tf.data.Dataset.from_tensor_slices((test_source_sequences, test_target_sequences))

# Batch and shuffle the training dataset
batch_size = 32
train_dataset = train_dataset.shuffle(buffer_size=10000).batch(batch_size, drop_remainder=True)
val_dataset = val_dataset.batch(batch_size, drop_remainder=True)
test_dataset = test_dataset.batch(batch_size, drop_remainder=True)

# Print dataset samples to verify
for source, target in train_dataset.take(1):
    print('Source batch shape:', source.shape)
    print('Target batch shape:', target.shape)


Source batch shape: (32, 72)
Target batch shape: (32, 68)


2024-11-10 07:40:47.823014: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-10 07:40:47.852079: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-10 07:40:47.852230: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [8]:
#
# Encoder
#
# Encoder with LSTM
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(
            self.enc_units,
            return_sequences=False,  # Set to True if you need the full sequence
            return_state=True,       # Set to True to get the last state (hidden and cell)
            recurrent_initializer="glorot_uniform",
        )

    def call(self, x):
        x = self.embedding(x)
        output, hidden_state, cell_state = self.lstm(x)  # Get output, hidden state, and cell state
        return hidden_state, cell_state  # Return both hidden and cell states


In [9]:
#
# Decoder
#
# Decoder with LSTM
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(
            self.dec_units,
            return_sequences=True,  # Set to True to return the full sequence
            return_state=True,      # Set to True to return the last hidden and cell states
            recurrent_initializer="glorot_uniform",
        )
        self.softmax = tf.keras.layers.Dense(vocab_size, activation="softmax")

    def call(self, x, hidden, cell):
        x = self.embedding(x)
        # Pass both hidden and cell states to the LSTM
        output, hidden_state, cell_state = self.lstm(inputs=x, initial_state=[hidden, cell])
        output = self.softmax(output)
        return output, hidden_state, cell_state  # Return output, hidden state, and cell state


In [10]:
# ========================================
# Create model
# ========================================

embedding_dim = 256
units = 1024  # LSTM/GRU dimensionality of the output space.

encoder = Encoder(source_vocab_size, embedding_dim, units, batch_size)
decoder = Decoder(target_vocab_size, embedding_dim, units, batch_size)

optimizer = tf.compat.v1.train.AdamOptimizer()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(reduction='none')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")
val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="val_accuracy")

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0)) # this masks '<pad>'
    """
    Example:

    real= tf.Tensor(
    [[21  1 44 0  0]   (jump !    <eos> <pad> <pad>)
    [ 17  9 24 2 44]   (i    go   there .     <eos>)
    [ 27  1 44 0  0]   (no   !    <eos> <pad> <pad>)
    [ 21 22 32 2 44]], (i    know you   .     <eos>)
    , shape=(4, 5), dtype=int64)

    where <pad> = 0.

    mask= tf.Tensor(
    [[True  True  True False False]
    [ True  True  True True  True ]
    [[True  True  True False False]
    [ True  True  True True  True ],
    shape=(4, 5), dtype=bool)
    """

    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

In [11]:
CHECKPOINT = True

In [12]:
# ========================================
# Training
# ========================================

checkpoint_path = (
    "./checkpointsLLSTM/seq2seq-sample-"
    + str(500)
    + "-embedding-"
    + str(embedding_dim)
    + "-hidden-"
    + str(units)
)

if CHECKPOINT == True:
    ckpt = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)
    ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=2)
    if ckpt_manager.latest_checkpoint:
        ckpt.restore(ckpt_manager.latest_checkpoint)
        print("Latest checkpoint restored!!")


@tf.function
def train(encoder, decoder, source_sentences, target_sentences, target_lang_tokenizer):
    with tf.GradientTape() as tape:
        h, c = encoder(source_sentences)

        dec_input = target_sentences[:, :-1]  # Decoder input
        expected_dec_output = target_sentences[:, 1:]  # Expected output

        predictions, _, _ = decoder(dec_input, h, c)
        loss = loss_function(expected_dec_output, predictions)
        train_accuracy(expected_dec_output, predictions)

    batch_loss = loss / int(target_sentences.shape[1])
    variables = encoder.variables + decoder.variables
    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss


@tf.function
def validate(encoder, decoder, source_sentences, target_sentences, target_lang_tokenizer):
    h, c = encoder(source_sentences)
    dec_input = target_sentences[:, :-1]  # Decoder input
    expected_dec_output = target_sentences[:, 1:]  # Expected output

    predictions, _, _ = decoder(dec_input, h, c)
    loss = loss_function(expected_dec_output, predictions)
    val_accuracy(expected_dec_output, predictions)  # Assuming you have val_accuracy defined

    return loss


# Set n_epochs at least 20 when you do training.
n_epochs = 1

# Prepare to store loss and accuracy
loss_history = []
accuracy_history = []
val_loss_history = []
val_accuracy_history = []

import time
for epoch in range(1, n_epochs + 1):
    start = time.time()

    total_loss = 0
    total_val_loss = 0
    train_accuracy.reset_states()
    val_accuracy.reset_states()  # Reset validation accuracy

    # Training loop
    for (batch, (source_sentences, target_sentences)) in enumerate(train_dataset):
        batch_loss = train(encoder, decoder, source_sentences, target_sentences, target_tokenizer)
        total_loss += batch_loss

        if batch % 100 == 0:
            print("Epoch {} Batch {} Loss {:.4f} Accuracy: {:.4f}".format(epoch, batch, batch_loss.numpy(), train_accuracy.result()))

    # Store the average loss and accuracy for this epoch
    avg_loss = total_loss / (batch + 1)
    loss_history.append(avg_loss)
    accuracy_history.append(train_accuracy.result().numpy())

    # Validation loop
    for (val_source_sentences, val_target_sentences) in val_dataset:
        val_loss = validate(encoder, decoder, val_source_sentences, val_target_sentences, target_tokenizer)
        total_val_loss += val_loss

    # Store the average validation loss and accuracy for this epoch
    avg_val_loss = total_val_loss / len(val_dataset)
    val_loss_history.append(avg_val_loss)
    val_accuracy_history.append(val_accuracy.result().numpy())

    if CHECKPOINT == True:
        ckpt_save_path = ckpt_manager.save()
        print("Saving checkpoint for epoch {} at {}".format(epoch, ckpt_save_path))

    print("Epoch {}/{} Loss {:.4f} Val Loss {:.4f}".format(epoch, n_epochs, avg_loss, avg_val_loss))
    print("Time taken for 1 epoch {:.4f} sec\n".format(time.time() - start))


Latest checkpoint restored!!


2024-11-10 07:40:49.131948: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-11-10 07:40:49.171056: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906


Epoch 1 Batch 0 Loss 0.0000 Accuracy: 0.0900
Epoch 1 Batch 100 Loss 0.0001 Accuracy: 0.1004
Epoch 1 Batch 200 Loss 0.0002 Accuracy: 0.1008
Epoch 1 Batch 300 Loss 0.0001 Accuracy: 0.1011
Saving checkpoint for epoch 1 at ./checkpointsLLSTM/seq2seq-sample-500-embedding-256-hidden-1024/ckpt-82
Epoch 1/1 Loss 0.0001 Val Loss 2.9007
Time taken for 1 epoch 16.8727 sec



In [13]:
# import matplotlib.pyplot as plt
# # Plotting Loss and Accuracy
# plt.figure(figsize=(14, 10))

# # Loss plot
# plt.subplot(2, 1, 1)
# plt.plot(loss_history, label='Training Loss', color='blue')
# plt.plot(val_loss_history, label='Validation Loss', color='orange')
# plt.title('Training and Validation Loss Over Epochs')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()

# # Accuracy plot
# plt.subplot(2, 1, 2)
# plt.plot(accuracy_history, label='Training Accuracy', color='green')
# plt.plot(val_accuracy_history, label='Validation Accuracy', color='red')
# plt.title('Training and Validation Accuracy Over Epochs')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()

# plt.tight_layout()
# plt.show()



In [14]:
target_tokenizer.word_index

{'sos': 1,
 'eos': 2,
 'saya': 3,
 'aku': 4,
 'tidak': 5,
 'tom': 6,
 'yang': 7,
 'di': 8,
 'ini': 9,
 'itu': 10,
 'dia': 11,
 'kamu': 12,
 'apa': 13,
 'akan': 14,
 'dengan': 15,
 'ke': 16,
 'ada': 17,
 'kau': 18,
 'untuk': 19,
 'bisa': 20,
 'dan': 21,
 'pergi': 22,
 'adalah': 23,
 'dari': 24,
 'kita': 25,
 'kami': 26,
 'ingin': 27,
 'anda': 28,
 'hari': 29,
 'orang': 30,
 'sudah': 31,
 'lebih': 32,
 'sangat': 33,
 'tahu': 34,
 'pada': 35,
 'mary': 36,
 'apakah': 37,
 'harus': 38,
 'dalam': 39,
 'rumah': 40,
 'makan': 41,
 'sedang': 42,
 'karena': 43,
 'mereka': 44,
 'banyak': 45,
 'bahasa': 46,
 'kalau': 47,
 'tahun': 48,
 'pak': 49,
 'suka': 50,
 'mana': 51,
 'baik': 52,
 'telah': 53,
 'lagi': 54,
 'pernah': 55,
 'punya': 56,
 'buku': 57,
 'sini': 58,
 'anak': 59,
 'seorang': 60,
 'benar': 61,
 'datang': 62,
 'melihat': 63,
 'hal': 64,
 'sebuah': 65,
 'jalan': 66,
 'jangan': 67,
 'menjadi': 68,
 'berapa': 69,
 'mau': 70,
 'kemarin': 71,
 'laki': 72,
 'malam': 73,
 'belajar': 74,
 'la

In [15]:
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction, sentence_bleu

In [16]:
# ========================================
# Translation
# ========================================

def evaluate(sentence, encoder, decoder, source_lang_tokenizer, target_lang_tokenizer):

    inputs = source_lang_tokenizer.texts_to_sequences([sentence])[0]

    inputs = tf.compat.v1.keras.preprocessing.sequence.pad_sequences(
        [inputs], maxlen=max_source_length, padding="post"
    )
    inputs = tf.convert_to_tensor(inputs)

    result = ""

    h, c = encoder(inputs)
    dec_input = tf.expand_dims([target_lang_tokenizer.word_index["sos"]], 0)

    for t in range(max_target_length):
        #
        # Greedy Search
        #
        predictions, h, c = decoder(dec_input, h, c)
        predicted_id = tf.argmax(predictions[0][0]).numpy()
        result += target_lang_tokenizer.index_word[predicted_id] + " "
        if target_lang_tokenizer.index_word[predicted_id] == "eos":
            return result

        dec_input = tf.expand_dims([predicted_id], 0)

    return result


def translate(sentence, encoder, decoder, source_lang_tokenizer, target_lang_tokenizer):
    result = evaluate(sentence, encoder, decoder, source_lang_tokenizer, target_lang_tokenizer)
    return result.capitalize()

"""
# for debug:
#sentence = "Su voz suena muy bello."
#sentence = "No nos gusta la lluvia."
sentence = "Nos gusta la lluvia."
result = translate(sentence, encoder, decoder, source_lang_tokenizer, target_lang_tokenizer)
print("Input    : {}".format(sentence))
print("Predicted: {}".format(result))

sys.exit()
"""
#
#
# Helper function to detokenize a sequence to text
def detokenize(sequence, tokenizer):
    return ' '.join([tokenizer.index_word.get(idx, '') for idx in sequence if idx != 0])

def calculate_bleu(reference, hypothesis):
    # Tokenize the sentences for BLEU score calculation

    # Use the sentence_bleu function to calculate the score
    bleu_score = sentence_bleu(reference, hypothesis, smoothing_function=SmoothingFunction().method1)
    return bleu_score
keys = np.arange(len(val_source_sequences))
keys = np.random.permutation(keys)[:10]
bleu_scores = []
for i in range(len(keys)):
    print("===== [{}] ======".format(i + 1))
    sentence = detokenize(val_source_sequences[i], source_tokenizer)
    result = translate(sentence, encoder, decoder, source_tokenizer, target_tokenizer)
    print("Input    : {}".format(sentence))
    print("Predicted: {}".format(result))
    correct_sentence = detokenize(val_target_sequences[i], target_tokenizer)
    print("Correct  : {}".format(correct_sentence))
    # Calculate and store the BLEU score
    bleu_score = calculate_bleu(correct_sentence, result)
    bleu_scores.append(bleu_score)
    print("BLEU Score: {:.4f}".format(bleu_score))
# Calculate the average BLEU score for the sample
average_bleu_score = np.mean(bleu_scores)
print("Average BLEU Score for the sample: {:.4f}".format(average_bleu_score))
# encoder.summary()
# decoder.summary()
# decoder.get_config()

===== [1] ======
Input    : in the western half the volume increased from million per year in 1998 to in and in 2011
Predicted: Ketika musim panas datang hawa menjadi panas eos 
Correct  : sos di bagian barat volume naik dari juta per tahun pada 1998 menjadi pada dan pada 2011 eos
BLEU Score: 0.0075
===== [2] ======
Input    : she obtained a scholarship from the department to study politics in the us
Predicted: Dia memutuskan untuk pergi ke amerika untuk belajar eos 
Correct  : sos kemudian ia mendapat beasiswa dari gereja untuk belajar di amerika serikat eos
BLEU Score: 0.0066
===== [3] ======
Input    : the bridge is the main connection between downtown and new and also carries the traffic on and through
Predicted: Kereta itu tiba sesuai jadwal eos 
Correct  : sos jembatan ini merupakan bagian dari jalan bebas hambatan di kota dan terletak di rute eropa eos
BLEU Score: 0.0104
===== [4] ======
Input    : in systems the used as for a hard are usually the received signal power and the r